In this part, we try to standarize/transform features and then drop parts of features.

We use dummy classifier as the bottom line. 

Compare the performance of different models on different sampling strategy

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from itertools import cycle

In [2]:
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import tensorflow as tf
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import (roc_auc_score,accuracy_score, classification_report,
confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve)
from sklearn.model_selection import train_test_split,GridSearchCV

/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
import pandas_profiling
import eli5

/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/matplotlib/__init__.py:1405: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
raw_data = pd.read_csv('~/Desktop/raw_data.csv')

In [22]:
#pandas_profiling.ProfileReport(raw_data)

## Feature prepocessing 

In [5]:
y = raw_data.iloc[:,-1].values
X = raw_data.iloc[:, :-1].values

In [6]:
feature_names = raw_data.columns.values[:-1]

#### 1.Standardize feature

In [7]:
scaler = StandardScaler()
Scale_X = scaler.fit_transform(raw_data.iloc[:, :-1])
Scaled_df = pd.DataFrame(X, columns = raw_data.columns.values[:-1])
#pandas_profiling.ProfileReport(Scaled_df)

In [8]:
Scaled_names = Scaled_df.columns.values

1.1 drop features of highly correlated and skewed

In [9]:
droped_Scale_X = Scaled_df.drop(['Domestic_Duration','International_Duration','T3_Flux','T11_Flux'], axis =1)

In [10]:
droped_Scaled_names = droped_Scale_X.columns.values

#### 2. log transformation

In [11]:
log_X = np.log(raw_data+1).iloc[:,:-1]
#pandas_profiling.ProfileReport(log_X)

In [12]:
log_names = log_X.columns.values

2.1 Transform features with high proportions of zero into 0-1 variables 

In [13]:
log_X_trans = log_X.copy()
f = lambda x: 1 if x>0 else 0
log_X_trans['Transfered_Duration'] = log_X_trans['Transfered_Duration'].apply(f)
log_X_trans['International_Duration'] = log_X_trans['International_Duration'].apply(f)
log_X_trans['T3_Flux'] = log_X_trans['T3_Flux'].apply(f)
log_X_trans['T11_Flux'] = log_X_trans['T11_Flux'].apply(f)

2.2 drop features

In [14]:
dropped_log_X_trans= log_X_trans.drop(['Domestic_Duration'],axis=1)

In [15]:
dropped_log_names = dropped_log_X_trans.columns.values

## Adjusted feature performances in machine learning models

In [16]:
# Build up Cross validation pipeline
def Pipeline_CV(X, y, model):
    # train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y, test_size=0.25)

    # preparing oversampling data 
    os = SMOTE(random_state=1) 
    os_X_train,os_y_train=os.fit_sample(X_train,y_train)
    
    print ('withoutsampling :')
    # model for withoutsampling data 
    #Use cross validation training data
    trained_model1 = cv_train_model (model, X_train, y_train, cv_folder=4)
    #Predict on the test set
    preds1 = trained_model1.predict_proba(X_test)[:,1]
    auc1 = roc_auc_score(y_test,preds1)
    print ('Test AUC score :',auc1)
    # model for oversampling data 
    
    print ('oversampling :')
    #Use cross validation training data 
    trained_model2 = cv_train_model (model, os_X_train, os_y_train, cv_folder=4)
    #Predict on the test set
    preds2 = trained_model2.predict_proba(X_test)[:,1]
    auc2 =roc_auc_score(y_test,preds2)
        
    print ('Test AUC score :',auc2)
    print ('')

# Use cross validation training data 
def cv_train_model (model, X_train, y_train, cv_folder):
    X_train = np.array (X_train)
    y_train = np.array (y_train).ravel()
    kfolder=StratifiedKFold(y_train, n_folds= cv_folder, shuffle=True, random_state=1)
    grand_auc =0.0
    i = 0
    for train_index, test_index in kfolder:
        X_trained, X_cv = X_train[train_index], X_train[test_index]
        y_trained, y_cv = y_train[train_index], y_train[test_index]
        model.fit(X_trained, y_trained)
        preds = model.predict_proba(X_cv)[:,1]
        auc=roc_auc_score(y_cv,preds)        
        grand_auc +=auc      
        i +=1
    
    grand_auc /= cv_folder
    print ('Validation AUC score is ',grand_auc)
    return model

In [17]:
print('DummyClassifier:')
model = DummyClassifier()
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)

DummyClassifier:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.4944000626566416
Test AUC score : 0.5115131578947368
oversampling :
Validation AUC score is  0.4920504385964912
Test AUC score : 0.46546052631578955

Standarized features
withoutsampling :
Validation AUC score is  0.4944000626566416
Test AUC score : 0.48930921052631576
oversampling :
Validation AUC score is  0.5019188596491229
Test AUC score : 0.5789473684210527

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.4933035714285714
Test AUC score : 0.48273026315789475
oversampling :
Validation AUC score is  0.4928728070175439
Test AUC score : 0.6118421052631579

Log transformed features
withoutsampling :
Validation AUC score is  0.4887609649122807
Test AUC score : 0.4876644736842105
oversampling :
Validation AUC score is  0.5052083333333334
Test AUC score : 0.5057565789473684

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.500587406015

The Adjusted features contributes to the performance of Dummy Classifier.

In [18]:
print('KNeighbors:')
model = KNeighborsClassifier()
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)

KNeighbors:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.46258223684210525
Test AUC score : 0.45619806094182824
oversampling :
Validation AUC score is  0.9318540320098492
Test AUC score : 0.5282635041551247

Standarized features
withoutsampling :
Validation AUC score is  0.553688909774436
Test AUC score : 0.4931613573407202
oversampling :
Validation AUC score is  0.9620748691905201
Test AUC score : 0.6127510387811634

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.45504385964912286
Test AUC score : 0.4578427977839335
oversampling :
Validation AUC score is  0.9340921004539858
Test AUC score : 0.5450571329639888

Log transformed features
withoutsampling :
Validation AUC score is  0.5108474310776943
Test AUC score : 0.53466932132964
oversampling :
Validation AUC score is  0.9657130078485687
Test AUC score : 0.6098943905817175

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.5141760651629073
Tes

As KNeighbors is measured by distance; it is found that the Transformed log transformed features perform best (0.6875) in KNeighbors algorithm. 

In [28]:
print('LogisticRegression:')
model = LogisticRegression(penalty='l1', C=0.009,class_weight='balanced', random_state =0)
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)
eli5.show_weights(model, feature_names=dropped_log_names)

LogisticRegression:
Unscaled or Untransformed features
withoutsampling :


/Users/zuochaozhao/anaconda/lib/python3.6/site-packages/sklearn/linear_model/base.py:340: RuntimeWarning: overflow encountered in exp
  np.exp(prob, prob)


Validation AUC score is  0.5952380952380952
Test AUC score : 0.6010214681440443
oversampling :
Validation AUC score is  0.7464664608341027
Test AUC score : 0.6248268698060943

Standarized features
withoutsampling :
Validation AUC score is  0.6282503132832079
Test AUC score : 0.5415512465373962
oversampling :
Validation AUC score is  0.6425883444906124
Test AUC score : 0.5419407894736842

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.5886591478696741
Test AUC score : 0.6084660664819944
oversampling :
Validation AUC score is  0.7020104743767313
Test AUC score : 0.612707756232687

Log transformed features
withoutsampling :
Validation AUC score is  0.5939458020050126
Test AUC score : 0.5953947368421052
oversampling :
Validation AUC score is  0.7249382021391197
Test AUC score : 0.640062326869806

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.5807879072681704
Test AUC score : 0.5895948753462603
oversampling :
Validation AUC scor

Weight?,Feature
+0.143,T7_Flux
+0.060,T5_Flux
+0.022,T8_Flux
+0.009,T6_Flux
+0.007,T10_Flux
-0.023,T1_Flux
-0.072,Called_Duration
-0.130,T4_Flux
-0.288,Visiting_Area_Diversity


The Dropped Log transformed features gives best performance (0.684) in LogisticRegression as the log transformed distrubution is not highly skewed

In [45]:
print('RandomForest:')
model = RandomForestClassifier(max_features= 0.5, max_depth =15, random_state =0)
print('Unscaled or Untransformed features')
Pipeline_CV(X, y, model)
print('Standarized features')
Pipeline_CV(Scale_X, y, model)
print('Dropped Standarized features')
Pipeline_CV(droped_Scale_X, y, model)
print('Log transformed features')
Pipeline_CV(log_X, y, model)
print('Transformed log transformed features')
Pipeline_CV(log_X_trans, y, model)
print('Dropped Log transformed features features')
Pipeline_CV(dropped_log_X_trans, y, model)
eli5.show_weights(model, feature_names=dropped_log_names)

RandomForest:
Unscaled or Untransformed features
withoutsampling :
Validation AUC score is  0.6053218984962405
Test AUC score : 0.5305574792243768
oversampling :
Validation AUC score is  0.9928710035780239
Test AUC score : 0.5026835180055402

Standarized features
withoutsampling :
Validation AUC score is  0.6053218984962405
Test AUC score : 0.5309903047091413
oversampling :
Validation AUC score is  0.9943293850030779
Test AUC score : 0.6415339335180055

Dropped Standarized features
withoutsampling :
Validation AUC score is  0.5397478070175438
Test AUC score : 0.56115824099723
oversampling :
Validation AUC score is  0.9920161732456141
Test AUC score : 0.576696675900277

Log transformed features
withoutsampling :
Validation AUC score is  0.5973919172932332
Test AUC score : 0.533154432132964
oversampling :
Validation AUC score is  0.9951884233610342
Test AUC score : 0.5872576177285318

Transformed log transformed features
withoutsampling :
Validation AUC score is  0.5776159147869675
Test 

Weight,Feature
0.2186 ± 0.1528,Visiting_Area_Diversity
0.0834 ± 0.0903,Roam_Ratio
0.0735 ± 0.0504,T7_Flux
0.0714 ± 0.0656,T5_Flux
0.0662 ± 0.0514,T4_Flux
0.0568 ± 0.0308,T2_Flux
0.0564 ± 0.0318,T9_Flux
0.0446 ± 0.0275,T1_Flux
0.0421 ± 0.0429,Called_Duration
0.0400 ± 0.0223,T10_Flux


The Standarized features gives best performance (0.6415) in RandomForest 